In [ ]:
import pandas as pd
import re

### Cleaning Yardi data
Removing from Locality: Numbers, "- ", Replacing / with _

In [ ]:
acs = pd.read_csv('ACS_Data_All_Tracts_and_Submarkets.csv')

In [ ]:
yardi = pd.read_csv('Yardi_Cleaned.csv')

In [ ]:
yardi = yardi[yardi['Locality'].str.contains('Total', case=False)==False]

In [ ]:
def clean_locality(text):
    text = re.sub(r'\d+', '', text)
    text = text.lstrip('- ')
    text = text.replace('/', '_')
    return text

yardi['Locality'] = yardi['Locality'].apply(clean_locality)

In [ ]:
yardi.to_csv('yardi_pee.csv')

### Finding unique submarkets from Yardi

In [ ]:
yardi = pd.read_csv('Yardi_Clean_NoTotals.csv')

In [ ]:
geoid_to_submarket = pd.read_csv('GEOIDs_to_submarkets.csv')
geoid_to_submarket = geoid_to_submarket.groupby(by='submarket').count().reset_index()

In [ ]:
merged_df = yardi.merge(geoid_to_submarket, left_on='Locality', right_on='submarket', how='outer', indicator=True)

non_matching_rows = merged_df[merged_df['_merge'] != 'both']

# Save non-matching rows to a new file
non_matching_rows.to_csv('non_matching_rows.csv', index=False)

In [ ]:
geoid_to_submarket.drop_duplicates(subset = 'submarket')
geoid_to_submarket

### Regression?

In [ ]:
import pandas as pd

In [ ]:
yardi = pd.read_csv('final_yardi.csv')

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Assume you have independent variables X and a dependent variable y
X = yardi[["Some_college_2010", "No_bedroom_2010"]]
y = yardi["Inventory_Aff_2010"]

# Create an instance of the LinearRegression class
reg = LinearRegression()

# Fit the model to the data
reg.fit(X, y)

# Print the coefficients of the model
#print(reg.coef_)

print(reg.predict([[43842184, 4125765]]))

### Finding cities for submarkets

#### Cleaning locality column

In [24]:
import pandas as pd

In [29]:
yardi = pd.read_csv('Yardi_with_cities.csv', low_memory=False)

In [30]:
# To lowercase
yardi['Locality'] = yardi['Locality'].str.lower()

In [31]:
# Deleting duplicates and blank rows
yardi = yardi.drop_duplicates()
yardi = yardi.dropna(how="all")

In [32]:
#yardi_cleaned.to_csv('pee.csv', index=False)

#### Manually filled in cities & re-upload

In [33]:
yardi = pd.read_csv('pee.csv')

In [36]:
yardi = yardi.dropna()

In [37]:
#yardi.to_csv('pee.csv', index=False)

#### Deleted extra cols & re-upload

In [1]:
import pandas as pd
import re

yardi = pd.read_csv('pee.csv')

def clean_locality(text):
    text = re.sub(r'\d+', '', text)
    text = text.lstrip('- ')
    text = text.replace('/', '_')
    return text

yardi['Locality'] = yardi['Locality'].apply(clean_locality)

In [2]:
yardi.to_csv('pee.csv', index=False)

#### Cleaning GEOIDs_to_submarkets.csv

In [1]:
import pandas as pd
import re

yardi = pd.read_csv('GEOIDs_to_submarkets.csv')

def clean_locality(text):
    text = text.replace(' ', '_')
    text = text.replace('-', '_')
    text = text.replace('/', '_')
    return text

yardi['submarket'] = yardi['submarket'].apply(clean_locality)

In [7]:
# Run this 2x
def clean_locality(text):
    text = text.replace('__', '_')
    return text

yardi['submarket'] = yardi['submarket'].apply(clean_locality)
yardi['submarket'] = yardi['submarket'].str.lower()

In [9]:
yardi.to_csv('GEOIDs_to_submarkets_CLEAN.csv')

### Merge Locality_and_City with overall dataset

In [1]:
import pandas as pd

In [2]:
acsyardi = pd.read_csv('acsyardicomplete.csv')
cities = pd.read_csv('Locality_and_city_CLEAN.csv')

In [63]:
# Need to replace spaces in Locality with underscores
'''
def clean_locality(text):
    text = text.replace(' ', '_')
    text = text.replace(' - ', '_')
    return text

cities['Locality'] = cities['Locality'].apply(clean_locality)

#cities.to_csv('Locality_and_city_CLEAN.csv')
'''

In [8]:
final = pd.merge(acsyardi, cities, left_on="Submarket", right_on="Locality", how="right")

In [9]:
final.to_csv("acsyardi_with_cities_RIGHTJOIN.csv")

### ~Merge GEOID, Submarket, City~

In [10]:
import pandas as pd

In [19]:
submarkets = pd.read_csv('submarkets_list_CLEAN.csv') # From combined DBFs
city = pd.read_csv('Locality_and_city_CLEAN.csv') # From Yardi excel file

In [23]:
df = pd.merge(submarkets, city, left_on='Submarket', right_on='Locality', how='inner')
df = df[['Submarket', 'Locality', 'City']]

In [24]:
df.to_csv('submarket_locality_city.csv')

In [25]:
geoid = pd.read_csv('GEOIDs_to_submarkets_CLEAN.csv')

In [29]:
df = pd.merge(geoid, df, left_on='submarket', right_on='Submarket', how='left')

In [30]:
df.to_csv('pee.csv')

### Merge GEOID, Submarket, City

In [1]:
import pandas as pd

In [2]:
geoid = pd.read_csv('GEOIDs_to_submarkets_CLEAN.csv') #Spatial data
city = pd.read_csv('Locality_and_city_CLEAN.csv') #Housing data

In [3]:
df = pd.merge(geoid, city, left_on='submarket', right_on='Locality', how='inner')

In [4]:
#df.to_csv('GEOID_submarket_city.csv')

In [7]:
df.groupby('Locality').count() #How many submarkets do we have spatial & housing data for?

,GEOID,submarket,City
Locality,,,
alamo_heights_central,5,5,5
arlington_central,11,11,11
arlington_east,11,11,11
arlington_north,17,17,17
arlington_south,28,28,28
...,...,...,...
west_end_downtown,36,36,36
west_end_fairlie_poplar_underground,20,20,20
west_side,13,13,13


In [20]:
acsyardi = pd.read_csv('acsyardi.csv')
city = pd.read_csv('submarkets_and_cities.csv')

In [21]:
city.drop_duplicates()

,Submarket,City
0,eubank_acres_south,Austin
1,eubank_acres_north,Austin
2,dessau,Austin
3,walnut_forest,Austin
4,pleasant_hill_east,Austin
...,...,...
318,clearwater_east,Clearwater
319,tampa_tampa_heights,Tampa
320,st_petersburg_east,St Petersburg
321,st_petersburg_north,St Petersburg


In [22]:
df = pd.merge(acsyardi, city, on='Submarket', how='inner')

In [23]:
df.to_csv('pee.csv')